In [70]:
#Imports 
#-----------------------------
# bring in the csv's to work with using pandas dataframe 
#-----------------------------


# pandas info http://pandas.pydata.org/


import numpy as np
import pandas as pd
import csv

keys= pd.read_csv("adjData/popData.csv")
adjList= pd.read_csv("adjData/nlist.csv")
isin=pd.read_csv("adjData/isin.csv")
extend=pd.read_csv("adjData/MDextend.csv")
keyList= keys[['GEOID']]
# two step join to get only maryland-maryland census tracts 
step= pd.merge(adjList,keyList,how='inner',left_on='NEIGHBOR_TRACTID',right_on='GEOID')
adjSet= pd.merge(step,keyList,how='inner',left_on='SOURCE_TRACTID',right_on='GEOID')
keys=pd.merge(keys,isin,how='inner',left_on='GEOID',right_on='GEOID')
keys=pd.merge(keys,extend,how='inner',left_on='GEOID',right_on='GEOID')

keys['FamilyProportion'] = keys['FamilyProportion'].convert_objects(convert_numeric=True)
keys['BlackPercentage'] = keys['BlackPercentage'].convert_objects(convert_numeric=True)

In [71]:
#PRECOMPUTE NEIGHBORS
#-----------------------------
# precompute number of neighbors so we do not leave any adjacent districts this is to make our lives way easier later 
#-----------------------------

# checked maryland values for no zero values so we can look for orphans right after they are created 

partners={}
for i in range(1,keyList.shape[0]):#
    key = keyList.iloc[i]['GEOID']
    value = adjSet[adjSet.SOURCE_TRACTID == key] # adjacent tracts for census tracts key in NEIGHBOR_TRACTID col 
    partners[key] = value.shape[0] 
   

In [72]:
# choose a starting discrict and start working 
# for test purposes we are going to make 1 district 24029950100
# could we precomput the number of districts that are adjcant to a disctric and each time they are added subract one -- much faster and
# we can search easily 

#globals
#feasibility constraint so we can make manual changes 
eps = np.int64(100000)
targetPop=np.int64(721694)-eps #721694
#function defs
def findDistrict(adjacent , tracts, pop, det,params ):
    #Check to see if we are done 
    
    if (pop+det >= targetPop).all():
        print ('-----Params----')
        print(params)
        print ('---------------')
        return tracts
    # choose next district based on selection method
    
    # get the tract in the adjacent set with highest value to be added  
    v=list(adjacent.values())
    k=list(adjacent.keys())
    # index of tract with most neighbors
    toAdd =k[v.index(max(v))].item()
    
    # once selection is made addtract will handle all of the set updates 
    adjacent,tracts,pop,params = addTract(adjSetl=adjacent,tract=toAdd,dist=tracts,pop=pop,params=params)
    # recursice call
    if  (len ( adjacent ) == 0 ):
        print ('-----Params In Error----')
        print(params)
        print ('------------------------')    
        return tracts
    #print (tracts)
    return findDistrict(adjacent,tracts,pop,det,params)

def addTract (adjSetl,tract,dist,pop,params):
    # we need to get the index of a given tract  
    index=(keys[keys.GEOID == tract])[['index']].iloc[0]
    if (keys.iloc[index]['IsIn'].any() == 0 ):
        #add tract to the district 
        dist.append(tract)
        #remove dist from adjset 
        adjSetl[tract] =-1
        keys.set_value(index, 13 , 1,takeable=True)
        # update pop
        pop=pop+(keys[keys.GEOID == tract])[['POP100']].iloc[0]  
        #update params 
        params=updateParams(tract,params)
        #update adj set for new adjacent 
        adjSetl=updateAdjSet(localAdjSet=adjSetl,index=tract,params=params)
        # deal with oprhaned districts 
        orphans=isOrphan(tract=tract)
        if (orphans != []):
            #deal with new oprhans we are going to create by adding this new tract 
            #being lazy / greedy and i am just going to add them this might get ~ weird 
            for val in orphans:
                #addTract(adjSetl=adjSetl,tract=val,dist=dist,pop=pop)
                afaf=val
    else: 
        #print ('trying to add already added')
        v=list(adjSetl.values())
        k=list(adjSetl.keys())
        toDel =k[v.index(max(v))]
        del adjSetl[toDel]
        
    return (adjSetl,dist,pop,params)

def isOrphan (tract):
    global partners 
    orphans=[]
    toRemove = adjSet[adjSet.SOURCE_TRACTID == tract] # adjacent tracts for census tracts key in NEIGHBOR_TRACTID col 
    for i in range(1,toRemove.shape[0]):
        key = toRemove.iloc[i]['NEIGHBOR_TRACTID']
        value=partners[key]
        if (value == 1): 
            orphans.append(key)
        partners[key]=value-1
    return orphans

def updateAdjSet(localAdjSet,index,params):
    # this is the code that is being rewrtitten to acomodate the function specified in our paper :)
    toAdd = adjSet[adjSet.SOURCE_TRACTID == index] # adjacent tracts for census tracts key in NEIGHBOR_TRACTID col
    for i in range(0,toAdd.shape[0]):#
        key = toAdd.iloc[i]['NEIGHBOR_TRACTID']  
        if  key in localAdjSet:
            if (localAdjSet[key]==-1):
                # do nothing this value is alredy been added to the district 
                b=1
            else:
                localAdjSet[key]=calcValue(key,params)
               
        else:
            localAdjSet[key]=calcValue(key,params)
    return localAdjSet
#calcValue 
#     toAdd  -- geoID of the tract we are calculating value of
#     params -- vector of data needed to preform the calculation from the adjset
#          params[0] -- average age of the district 
#          params[1] -- std deviation of age
#          params[2] -- mean Family Proportion
#          params[3] -- std deviation of Family Proportion
#          params[4] -- average %black
#          params[5] -- std dev of %black
#          params[6] -- number of tracts in dist 
#
#  the function is of the form Value = 17*(age_diff/sigma) + 5*(household_diff/sigma)^2 + 1*(race_diff/sigma)
def calcValue(toAdd,params):
    age   = keys[keys.GEOID == toAdd]['Medianage']
    house = keys[keys.GEOID == toAdd]['FamilyProportion']
    race  = keys[keys.GEOID == toAdd]['BlackPercentage']
    val=abs(17*(age.iloc[0]-params[0])/params[1])+abs(5*(house.iloc[0]-params[2])/params[3])+abs((age.iloc[0]-params[4]) /params[5])
    return val

def updateParams(toAdd,params):
    age   = keys[keys.GEOID == toAdd]['Medianage']
    house = keys[keys.GEOID == toAdd]['FamilyProportion']
    race  = keys[keys.GEOID == toAdd]['BlackPercentage']
    #when were here we might as well update the params since they will only be used here
    # update the averages 
    x1barold=params[0]
    x2barold=params[2]
    x3barold=params[4]
    params[0]=(params[0]*params[6]+age.iloc[0])/(params[6]+1)
    params[2]=(params[2]*params[6]+house.iloc[0])/(params[6]+1)
    params[4]=(params[4]*params[6]+race.iloc[0])/(params[6]+1)
    params[6]=params[6]+1
    if  params[6] == 1:
        params[1]=.10
        params[3]=.10
        params[5]=.10
        print('started')
    else:
        n=params[6]
        #update the std devs 
        # linkerino for the matherino http://bit.ly/1YI3W4R
        params[1]=pow(abs(((n-2)*pow(params[1],2)+(age.iloc[0]-params[0])*(age.iloc[0]-x1barold))/(n-1)),.5)
        params[3]=pow(abs(((n-2)*pow(params[3],2)+(house.iloc[0]-params[3])*(house.iloc[0]-x2barold))/(n-1)),.5)
        params[5]=pow(abs(((n-2)*pow(params[3],2)+(race.iloc[0]-params[3])*(race.iloc[0]-x3barold))/(n-1)),.5)
    return params
    

In [73]:
params=[0,0,0,0,0,0,0]
localAdjSet={}
age   = keys[keys.GEOID == 24001000100]['Medianage']
house = keys[keys.GEOID == 24001000100]['FamilyProportion']
race  = keys[keys.GEOID == 24001000100]['BlackPercentage']
    #when were here we might as well update the params since they will only be used here
    # update the averages 
x1barold=params[0]
x2barold=params[2]
x3barold=params[4]

params[0]=(params[0]*params[6]+age) / ( params[6]+1)
params[2]=(params[2]*params[6]+house)/ ( params[6]+1)
params[4]=(params[4]*params[6]+race)/ ( params[6]+1)
params[1]=.1
params[3]=.1
params[5]=.1
params[6]=params[6]+1
toAdd = adjSet[adjSet.SOURCE_TRACTID == 24001000100] # adjacent tracts for census tracts key in NEIGHBOR_TRACTID col
val=abs(17*(age.iloc[0]-params[0])/params[1])+abs(5*(house.iloc[0]-params[2])/params[3])+abs((age.iloc[0]-params[4]) /params[5])
print(val)

0    452.905863
dtype: float64


In [74]:
#5


startTractId5=24001000100
#24001000100
e={}
p=updateParams(24001000100,[0,0,0,0,0,0,0])
print(p)
fith= [startTractId5]
fithStartPop=(keys[keys.GEOID == startTractId5])[['POP100']].iloc[0]
locAdjSet5=updateAdjSet(localAdjSet=e,index=startTractId5,params=p)
fith=findDistrict(adjacent=locAdjSet5,tracts=fith,pop=fithStartPop,det=0,params=p)



started
[45.299999999999997, 0.1, 0.71240971799999997, 0.1, 0.0094136630000000009, 0.1, 1]
-----Params----
[44.331645569620264, 5.4383058504703881, 0.70486530003797465, 0.17586997478487101, 0.062993004797468291, 0.17526197293586981, 158]
---------------


In [75]:
p=[0,1,0,0,0,0]
print(p[1])

1


In [76]:
print (fith)

[24001000100, 24001000200, 24043010700, 24001000100, 24043010600, 24001000300, 24001001502, 24001000400, 24001001000, 24001001401, 24001001100, 24001001200, 24001001402, 24001000800, 24001000600, 24001000700, 24001000500, 24001001300, 24001001900, 24001002000, 24001002200, 24001001700, 24001001600, 24001001503, 24023000300, 24023000500, 24023000600, 24043010500, 24001002100, 24023000200, 24023000400, 24043010801, 24043010802, 24023000100, 24043000301, 24043010300, 24043010200, 24043010900, 24043000602, 24043000200, 24043000400, 24043011400, 24021770700, 24021751203, 24021752602, 24021751301, 24043011302, 24021751302, 24021750802, 24021767500, 24043011000, 24043010400, 24043011500, 24043000800, 24023000700, 24001001800, 24043000100, 24021752601, 24043000601, 24021751701, 24021752801, 24043010100, 24021751201, 24013501002, 24043000302, 24021750506, 24021750801, 24043001002, 24021751702, 24013509002, 24013503000, 24021753001, 24043011202, 24013514201, 24021752802, 24021751600, 24021740200

In [77]:
    a={};
    #get the first District 

    startTractId1 = 24021750100
    p=updateParams(startTractId1,[0,0,0,0,0,0,0])
    first= [startTractId1]
    firstStartPop=(keys[keys.GEOID == startTractId1])[['POP100']].iloc[0]
    locAdjSet1=updateAdjSet(localAdjSet=a,index=startTractId1,params=p)
    first= findDistrict(adjacent=locAdjSet1,tracts=first,pop=firstStartPop,det=0,params=p)

    # now the second 
    foo={}
    startTractId2 = 24047990000
    p=updateParams(startTractId2,[0,0,0,0,0,0,0])
    second= [startTractId2]
    secondStartPop=(keys[keys.GEOID == startTractId2])[['POP100']].iloc[0]
    locAdjSet2=updateAdjSet(localAdjSet=foo,index=startTractId2,params=p)
    second = findDistrict(adjacent=locAdjSet2,tracts=second,pop=secondStartPop,det=0,params=p)

    # 3 
    c={}
    startTractId3 = 24033805909
    p=updateParams(startTractId3,[0,0,0,0,0,0,0])
    third= [startTractId3]
    thirdStartPop=(keys[keys.GEOID == startTractId3])[['POP100']].iloc[0]
    locAdjSet3=updateAdjSet(localAdjSet=c,index=startTractId3,params=p)
    thrid = findDistrict(adjacent=locAdjSet3,tracts=third,pop=thirdStartPop,det=0,params=p)

    # 4
    startTractId4=24033803509
    d={}
    p=updateParams(startTractId4,[0,0,0,0,0,0,0])
    fourth= [startTractId4]
    fourthStartPop=(keys[keys.GEOID == startTractId4])[['POP100']].iloc[0]
    locAdjSet4=updateAdjSet(localAdjSet=d,index=startTractId4,params=p)
    fourth = findDistrict(adjacent=locAdjSet4,tracts=fourth,pop=fourthStartPop,det=0,params=p)

    #6 
    startTractId6=24510100100
    f={}
    p=updateParams(startTractId6,[0,0,0,0,0,0,0])
    sixth= [startTractId6]
    sixthStartPop=(keys[keys.GEOID == startTractId6])[['POP100']].iloc[0]
    locAdjSet6=updateAdjSet(localAdjSet=f,index=startTractId6,params=p)
    sixth= findDistrict(adjacent=locAdjSet6,tracts=sixth,pop=sixthStartPop,det=0,params=p)
    #7
    startTractId7=24027605505
    g={}
    p=updateParams(startTractId7,[0,0,0,0,0,0,0])
    sev= [startTractId7]
    sevStartPop=(keys[keys.GEOID == startTractId7])[['POP100']].iloc[0]
    locAdjSet7=updateAdjSet(localAdjSet=g,index=startTractId7,params=p)
    sev = findDistrict(adjacent=locAdjSet7,tracts=sev,pop=fithStartPop,det=0,params=p)

    #8
    startTractId8=24019970100
    h={}
    p=updateParams(startTractId8,[0,0,0,0,0,0,0])
    eight= [startTractId8]
    eightStartPop=(keys[keys.GEOID == startTractId8])[['POP100']].iloc[0]
    locAdjSet8=updateAdjSet(localAdjSet=h,index=startTractId8,params=p)
    eight = findDistrict(adjacent=locAdjSet8,tracts=eight,pop=eightStartPop,det=0,params=p)





started
-----Params----
[43.581379310344829, 7.0817840090585857, 0.71781046305517204, 0.19657233447064679, 0.10470991342758627, 0.19610505093156799, 145]
---------------
started
-----Params----
[40.64050000000001, 7.744669734500139, 0.61905683887999963, 0.13695202471689585, 0.25105753154499999, 0.13700044917077769, 200]
---------------
started
-----Params----
[37.499999999999957, 5.7766655489880074, 0.68614972497986548, 0.18138800745285269, 0.44520770673154331, 0.18315962585362822, 149]
---------------
started
-----Params----
[35.979310344827582, 4.557364256400545, 0.70487831098620712, 0.097846003727676586, 0.63377009172413779, 0.10466165293512222, 145]
---------------
started
-----Params----
[38.998265895953764, 4.8840327645468706, 0.62347833434682032, 0.22145259426729613, 0.42631020235838146, 0.22619188743882634, 173]
---------------
started
-----Params In Error----
[35.091891891891891, 3.8499146421502237, 0.66017382380180201, 0.08762009694027395, 0.41055046789189209, 0.0858389474017

In [78]:
#generate the CSV File for raw output 
f = open("mdComplexRaw.csv",'w')
f.write('GEOID' +','+'color' + '\n' )
for row in fith:
    f.write( str(row) +','+'5' + '\n' )
for row in first:
    f.write( str(row) +','+'1' + '\n' )
for row in second:
    f.write( str(row) +','+'2' + '\n' )
for row in thrid:
    f.write( str(row) +','+'3' + '\n' )
for row in fourth:
    f.write( str(row) +','+'4' + '\n' )
for row in sixth:
    f.write( str(row) +','+'6' + '\n' )
for row in sev:
    f.write( str(row) +','+'7' + '\n' )
for row in eight:
    f.write( str(row) +','+'8' + '\n' )
f.close()


In [79]:
#begin the recursive fixing 
#pd.set_option('display.max_rows', 1000)
import math as math
# now we have the subset for which to work with 
lo=keys[keys.IsIn == 0 ].GEOID  

def fix (tracts,toWrite):
    returnFlag=0;
    for row in tracts:
        toCheck=adjSet[adjSet.SOURCE_TRACTID == row].NEIGHBOR_TRACTID
        adjVal=0
        n=toCheck.shape
        scalar=n[0];
        adjacent={}
        for elm in toCheck:
            if elm in first:
                key=1
            elif elm in second:
                key=2
            elif elm in thrid:
                key=3
            elif elm in fourth:
                key=4
            elif elm in fith:
                key=5
            elif elm in fourth:
                key=6
            elif elm in sev:
                key=7
            elif elm in eight:
                key=8
            # deal with the dictionary values
            elif elm in toWrite:
                key=toWrite[elm] 
            else:
                key=-1
            
            if  key in adjacent and key != -1:
                adjacent[key]=adjacent[key]+1
            elif key != -1:
                adjacent[key]=1     
        #compute the value and update 
        # get adjacent with most edge 
    
        v=list(adjacent.values())
        k=list(adjacent.keys())
        if v == []:
            writeme=0
        # index of tract with most neighbors
        else:
            writeme =k[v.index(max(v))]
        # now we have the district to put our tract in 
        if (writeme != 0 ):
            toWrite[row]=writeme
            # set the tracts to is in in keys. 
            toUpdate=keys[keys.GEOID == row].index
            keys.set_value(toUpdate[0], 13 , 1,takeable=True)
        else:
            print('island found')
            returnFlag=1;
    #end of loop ----
    toWrite[24023000100]=5
    toWrite[24023000200]=5
    return toWrite
    #fix(lo,toWrite)
# end of function 


toWrite ={}
fix(lo,toWrite)
print('donezo')

island found
island found
island found
island found
island found
island found
island found
island found
island found
island found
island found
island found
island found
island found
island found
island found
island found
island found
island found
island found
island found
island found
island found
island found
island found
island found
island found
island found
island found
island found
island found
island found
donezo


In [80]:
# take a second pass for this one 
lo=keys[keys.IsIn == 0 ].GEOID 
fix(lo,toWrite)
print('donezo')

island found
island found
island found
island found
island found
island found
island found
island found
island found
island found
island found
island found
island found
island found
donezo


In [83]:
lo=keys[keys.IsIn == 0 ].GEOID
for row in lo: 
    print(row)
    toWrite[row]=6

24005491401
24015031302
24025301302
24025302802
24510120400
24510130200
24510151300
24510270701
24510270801
24510270802
24510270902
24510270903
24510271001
24510271102


In [84]:
#generate the CSV File for filtered output 
f = open("mdComplexMod.csv",'w')
f.write('GEOID' +','+'color' + '\n' )
for row in fith:
    f.write( str(row) +','+'5' + '\n' )
for row in first:
    f.write( str(row) +','+'1' + '\n' )
for row in second:
    f.write( str(row) +','+'2' + '\n' )
for row in thrid:
    f.write( str(row) +','+'3' + '\n' )
for row in fourth:
    f.write( str(row) +','+'4' + '\n' )
for row in sixth:
    f.write( str(row) +','+'6' + '\n' )
for row in sev:
    f.write( str(row) +','+'7' + '\n' )
for row in eight:
    f.write( str(row) +','+'8' + '\n' )
for key in toWrite.keys():
     f.write( str(key) +','+ str(toWrite[key]) + '\n' )
f.close()